<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Вам нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.


Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

In [1]:
pip install catboost

In [2]:
pip install lightgbm

In [19]:
import pandas as pd
import warnings
import numpy as np
warnings.simplefilter('ignore')
from scipy import stats as st
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg' 
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from sklearn.utils import shuffle
from sklearn.model_selection import (
    GridSearchCV, 
    RandomizedSearchCV,
    train_test_split
)
import lightgbm
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (StandardScaler)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
np.random.seed(0)

import catboost
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostRegressor, cv
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler
)
from pandas_profiling import ProfileReport
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit

In [4]:
try :
    data  = pd.read_csv('/datasets/taxi.csv', index_col=[0], parse_dates=[0])
    
except:
    data  = pd.read_csv('taxi.csv', index_col=[0], parse_dates=[0])
    
display(data.head(10))


In [5]:
data=data.resample('1H').sum()

## Анализ

In [6]:
profile = ProfileReport(data,
                        title='Pandas Profiling Report')

In [7]:
profile.to_widgets()

Видим что пропусков нет распределение данных похоже на нормальное . дубликатов скорее всего нет


In [8]:
figsize = (18,10)
data.plot(figsize=figsize)

In [9]:
data['2018-06-15':'2018-06-30'].plot(figsize=figsize)

In [10]:
data.sort_index(inplace=True)


decomposed = seasonal_decompose(data)

decomposed.plot() 
    


Видим что есть явный тренд на повышение со временем

In [11]:


decomposed_data = seasonal_decompose(data['2018-07-14':'2018-07-31'])
decomposed_data.trend.plot(figsize=figsize)

In [12]:
decomposed_data.seasonal.plot(figsize=figsize)

In [13]:
decomposed_data.resid.plot(figsize=figsize)

Вывод

Дисперсия со временем увеличичвается

Тренд на рост со временем 
Видно что в раннее утро кол-во заказов резко падает
Временной ряд в не является стационарным, так как подвержен тренду, который меняется во времени.
lag примем как 24 часа

rolling_mean примем как сутки то есть 48 часа
P.S. Подсмотрел параметры в у других,а как найти эти параметры, как они влияют на работу? 

In [14]:
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()


make_features(data,24,48)

data=data.dropna()

In [15]:
features=data.drop('num_orders',axis=1)
target=data['num_orders']
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.1, random_state=12345, shuffle=False)

print(features_train.shape)
print(features_test.shape)


## Обучение

In [16]:
model_cat=CatBoostRegressor()
model_rfr =  make_pipeline(RandomForestRegressor())
model_lr = make_pipeline(LinearRegression())
model_LGBM= LGBMRegressor() 
n_iter_search = 12 

In [20]:
parameters_cat = {'learning_rate':[0.1,0.6,0.8,1],
                  'iterations':[100,200],
                  'random_state':[12345],
                  'depth': [4, 6,10,20,30]
                 }
parameters_lr = {'linearregression__normalize':[False]}
parameters_rfr = {'randomforestregressor__n_estimators':[5,10,50,100],
                  'randomforestregressor__max_depth':[3,8,10,12,15,20],
                  'randomforestregressor__random_state':[12345]}
parameters_LGBM={'num_leaves':[31, 100, 200], 
                'learning_rate':[0.1, 0.3, 0.5,0.6,0.8,1],
                'random_state':[12345],
                'l2_leaf_reg': [1, 3, 5, 7, 9]}

In [21]:
tscv = TimeSeriesSplit(n_splits=8)


In [22]:
%%time
regressor = LinearRegression()


cv_RMSE_LR = (cross_val_score(regressor, 
                             features_train, 
                             target_train, 
                             cv=tscv, 
                              scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('Mean RMSE from CV of LinearRegression =', cv_RMSE_LR)

In [23]:
%%time

regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])

cv_RMSE_LR_S = (cross_val_score(pipeline,
                               features_train, 
                                target_train, 
                                cv=tscv, 
                                 scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('Mean RMSE from CV of LinearRegression =', cv_RMSE_LR_S)

Видим что стандартизация признаков не влияет на эффективность модели

In [24]:
model=LinearRegression()
random_search = RandomizedSearchCV(model_lr, 
                                   param_distributions=parameters_lr, 
                                   n_iter=10, 
                                   cv=tscv, 
                                   scoring = 'neg_mean_squared_error',
                                      ) 
random_search.fit(features_train, target_train) 
lr_final =random_search.best_score_
print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка : {}".format(random_search.best_score_))


In [25]:
print('Лучший результат в модели LinearRegression',((random_search.best_score_)* -1) ** 0.5)

In [26]:

random_search = RandomizedSearchCV(model_cat, 
                                   param_distributions=parameters_cat, 
                                   n_iter=10, 
                                   cv=tscv, 
                                   verbose=True,scoring = 'neg_mean_squared_error'
                                   
                                  )
                                   
                                     
random_search.fit(features_train, target_train) 
Cat_final =random_search.best_score_
print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка : {}".format(random_search.best_score_))


In [27]:
print('Лучший результат в модели cat',((random_search.best_score_)* -1) ** 0.5)

In [28]:
%%time
random_search = RandomizedSearchCV(model_LGBM, 
                                   param_distributions=parameters_LGBM, 
                                   n_iter=10, 
                                   cv=tscv, 
                                   scoring = 'neg_mean_squared_error',
                                   
                                      ) 
random_search.fit(features_train, target_train) 
LGBM_final =random_search.best_score_
print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка : {}".format(random_search.best_score_))

In [29]:
print('Лучший результат в модели LGBM',((random_search.best_score_)* -1) ** 0.5)

In [30]:
%%time
random_search_rfr_rougher = RandomizedSearchCV(model_rfr, param_distributions=parameters_rfr , n_iter=n_iter_search, cv=tscv, n_jobs=-1,scoring = 'neg_mean_squared_error')
random_search_rfr_rougher.fit(features_train, target_train)
rfr_rougher =random_search_rfr_rougher.best_score_
# Вывод результатов
print("Лучшие параметры: {}".format(random_search_rfr_rougher.best_params_)) 
print("Лучшая оценка : {}".format(random_search_rfr_rougher.best_score_))


In [31]:
print('Лучший результат в модели rfr',((random_search_rfr_rougher.best_score_)* -1) ** 0.5)

In [32]:
index = ['LinearRegression',
         
         
         'CatBoostRegressor ',
         
         'LGBMRegressor',
         'RandomForestRegressor'
        ]
d = {'RMSE модели ':[ 93.53,
                                           19.63,
                                                
                                              
                                                18.89,
                                              19.32]
   }
scores_data = pd.DataFrame(data=d, index=index)

In [33]:
display(scores_data)

Лучшие результаты показала модель LGBMRegressor	. Выбираем ее и протестируем на тестовых данных

## Тестирование

In [34]:
%%time
model = LGBMRegressor(learning_rate=0.1, 
                      num_leaves=100, 
                      random_state=12345,l2_leaf_reg= 5)
model.fit(features_train, target_train)
target_predict = model.predict(features_test)
final_RMSE_LGBMR = mean_squared_error(target_test, target_predict) ** 0.5
print( 'итоговый результат модели на тестовой выборке' ,final_RMSE_LGBMR)

In [35]:
mp = pd.Series(model.feature_importances_,
                         features_train.columns)

fig, ax = plt.subplots(figsize=(10,10))
mp.plot.bar(ax=ax)
ax.set_title("Важность признаков")
ax.set_ylabel('Важность')
fig.tight_layout()


Видно что наиболее важны сдвиг на 1-2 часа и на 18-20 часов. Впрочем причины особо непонятны

In [36]:
plt.figure(figsize=figsize)
plt.plot(target_test.index, 
         target_test, 
         'b-', 
         target_test.index, 
         target_predict,
         'r-');

При выполнении проекта проделана следующая работа:

На первом этапе произведена загрузка данных и их подготовка для обучения моделей.
За время наблюдений наметился определенный тренд на общее увеличение заказов такси с течением времени.На графиках явно видна суточная сезонность. Ночью количество заказов стремится к нулю, в то время как вечерний час пик - момент самого сильного спроса на услуги такси.
На третьем этапе к датасету были добавлены дополнительные признаки.после чего были выяснены оптимальные гиперпараметры для моделей
На четвертом этапе проведено тестирование всех моделей, каждой из них удалось достичь требуемого показателя метрики RMSE. 
Далее была выбрана и протестирована модель LGBMRegressor итоговый результат модели на тестовой выборке 42.86  что проходит отбор

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Данные проанализированы
- [ ]  Модель обучена, гиперпараметры подобраны
- [ ]  Качество моделей проверено, выводы сделаны
- [ ]  Значение *RMSE* на тестовой выборке не больше 48